In [2]:
# run ame on differential peaks

In [3]:
import os
import numpy as np
import pandas as pd
import subprocess as sp
from importlib import reload 
import pybedtools as pbt
pbt.set_bedtools_path("/localenv/rabbani/anaconda/miniconda3/envs/matplotlib/bin/")
pbt = reload(pbt)

In [4]:
# get the diffpeaks sequences
df = pd.read_csv(os.path.join("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diff_peaks.bed"),
                 sep = "\t", header = None)
df_fasta = pbt.BedTool.from_dataframe(df)
df_fasta = df_fasta.sequence(fi="/data/repository/organisms/GRCm38_ensembl/genome_fasta/genome.fa",
                             fo = "diffpeaks.fasta")    

In [4]:
# call ame with shuffle bg

In [5]:
cmd = "module load meme;"
cmd += "ame --control '--shuffle--' "
cmd += "-oc diffpeaks_motifs/ "
cmd += " diffpeaks.fasta "
cmd += " /data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_MOUSE_mono_meme_format.meme "
print(cmd)
sp.check_output(cmd, shell = True)


module load meme;ame --control '--shuffle--' -oc diffpeaks_motifs/  diffpeaks.fasta  /data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_MOUSE_mono_meme_format.meme 


Added /data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_MOUSE_mono_meme_format.meme to motif_sources which now has 1 file names.
Motif file name is /data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_MOUSE_mono_meme_format.meme.
Writing results to output directory 'diffpeaks_motifs/'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_MOUSE_mono_meme_format.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (324).
MOTIF: 1 SEQ: 1620/1620
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 529 SEQ: 1620/1620


b''

In [5]:
# make a meme file of ame.tsv

In [5]:
my_motifs = pd.read_csv("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/"\
                        "diffpeaks_motifs/ame.tsv", usecols = ["motif_ID"], sep = "\t")
my_motifs["motif_ID"].astype(str)

0      RFX3_MOUSE.H11MO.0.C
1      RFX2_MOUSE.H11MO.0.A
2      RFX1_MOUSE.H11MO.1.A
3      RFX6_MOUSE.H11MO.1.C
4      RFX3_MOUSE.H11MO.1.C
               ...         
122    ZEP2_MOUSE.H11MO.0.D
123    BATF_MOUSE.H11MO.0.A
124                     nan
125                     nan
126                     nan
Name: motif_ID, Length: 127, dtype: object

In [17]:
# make a meme file of found motifs , this is needed as an input for tobias
output = open("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs/ame.meme","w")
f = open("/data/repository/misc/Motif/HOCOMOCO/HOCOMOCOv11_full_MOUSE_mono_meme_format.meme", "r")
output.write("MEME version 4\n\nALPHABET= ACGT\n\nstrands: + -\n\nBackground letter frequencies\n\n"\
             "A 0.25 C 0.25 G 0.25 T 0.25\n\n")
lines = f.readlines()
write_it = False
for line in lines:
    this_line = line.rstrip()
    if any(motif in this_line for motif in my_motifs["motif_ID"].astype(str)):
        if this_line.startswith("MOTIF"):
            write_it = True
        elif this_line.startswith("URL"):
            output.write(this_line+"\n")
            output.write("\n")
            write_it = False
    if write_it == True:
        output.write(this_line+"\n")

In [16]:
# cluster them

In [ ]:
%%bash
conda activate /localenv/deboutte/anaconda/miniconda3/envs/TOBIASenv;
TOBIAS ClusterMotifs -m ame.meme -t 0.4 -a meme --dist_method seqcor --clust_method complete -o tobias_cluster

In [18]:
cmd = "module load meme;"
cmd += "ame --control '--shuffle--' "
cmd += "-oc diffpeaks_tobias_motifs/ "
cmd += " diffpeaks.fasta "
cmd += "/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs/tobias_cluster/motif_comparison_consensus_motifs.meme "
print(cmd)
sp.check_output(cmd, shell = True)

module load meme;ame --control '--shuffle--' -oc diffpeaks_tobias_motifs/  diffpeaks.fasta /data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs/tobias_cluster/motif_comparison_consensus_motifs.meme 


Added /data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs/tobias_cluster/motif_comparison_consensus_motifs.meme to motif_sources which now has 1 file names.
Motif file name is /data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs/tobias_cluster/motif_comparison_consensus_motifs.meme.
Writing results to output directory 'diffpeaks_tobias_motifs/'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diffpeaks_motifs/tobias_cluster/motif_comparison_consensus_motifs.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (324).
MOTIF: 1 SEQ: 1620/1620
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorte

b''